# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,27.153038,0.967306,0.326785,0.046083,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.976682,0.977538,0.974843,0.968553,0.970350,0.973594,0.003538,1
8,63.446545,4.069158,0.536442,0.056623,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978475,0.977538,0.973944,0.967655,0.970350,0.973594,0.004131,1
11,47.699498,5.444906,0.352390,0.046983,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977578,0.977538,0.973944,0.968553,0.969452,0.973415,0.003846,3
10,28.119654,0.505303,0.318610,0.018918,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977578,0.973944,0.973944,0.969452,0.968553,0.972696,0.003306,4
5,36.997295,1.700212,0.374963,0.045407,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.975785,0.974843,0.972147,0.966757,0.972147,0.972337,0.003143,5


In [4]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,50.945392,0.372854,0.452621,0.016955,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977578,0.974843,0.974843,0.970350,0.973046,0.974133,0.002382,1
7,26.318685,0.214410,0.304112,0.009305,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977578,0.974843,0.973046,0.968553,0.968553,0.972517,0.003543,2
11,51.133512,6.387495,0.400886,0.099912,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976682,0.973944,0.973046,0.967655,0.971249,0.972517,0.002997,2
10,28.618524,0.472090,0.326253,0.024705,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977578,0.975741,0.971249,0.966757,0.968553,0.971978,0.004125,4
4,22.290826,0.593078,0.288410,0.047062,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.975785,0.975741,0.971249,0.965858,0.967655,0.971259,0.004068,5
